In [ ]:
import pyodbc
import csv
#import mysql.connector as conn
try:
    #connect to MySql work bench
    #conn = mysqlConnector.connect(host='localhost',user='root',passwd='root')

    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=IN16993107W1\MSSQLSERVER01;'
                          'Database=Master;'
                          'Trusted_Connection=yes;')
    cursor = conn.cursor()
    
    """Create database"""
    try:
        cursor.execute("use cardataset")
    except:
        cursor.execute("create database cardataset")
        cursor.execute("use cardataset")

    """Read the file to csv"""
    file = open('car.data')
    csvreader = csv.reader(file)
    header = next(csvreader)

    """Create table dynamically"""
    query =''
    try:
        cursor.execute('select * from CarTable')
    except:    
        query = 'CREATE TABLE CarTable ('
        for item in header:
            query = query + item + ' varchar(50),'    
            i=i+1
        query = query[:-1] + ')' 
        cursor.execute(query)


    """Insert the data in to table"""
    strValues = ''
    dataCount = 0
    for row in csvreader:
        dataCount = dataCount + 1
        strQuery = 'insert into CarTable values ('
        for item in row: 
            strValues =strValues +  "'" + item + "',"
        strValues = strValues[:-1]
        strQuery = strQuery + strValues + ')'
        cursor.execute(strQuery)
        strValues= ''
        strQuery=''  
    conn.commit()
    
    """Check whether the record count matches"""
    cursor.execute("select count(*) from CarTable")    
    col=cursor.fetchall()
    dbRecordCount = 0
    for x in col:
        dbRecordCount = x[0]       
    
    if(dataCount == dbRecordCount):
        print(str(dataCount) + ' records inserted.')
    else:
        print('Record count insetred(' + str(dbRecordCount) +') does not match with the file record Count('+ str(dataCount) +').')
    
    print ("Data loading completed successfully..")
    
    """Group by Col1(buying), and getting the record count for each item"""
    cursor.execute("SELECT buying, COUNT(buying) FROM CarTable GROUP BY buying")
    col=cursor.fetchall()
    print('Group by Col1(buying), and getting the record count for each item')
    for x in col:
        print(x)
    
    """Filter a record where col3(doors) is 3"""
    cursor.execute("SELECT * FROM CarTable where doors = '4'")
    col=cursor.fetchall()
    print('Filter a record where col3(doors) is 4')
    for x in col:
        print(x)
        
        
    """Update col3(doors) to 8 where ever the value is 2"""
    cursor.execute("UPDATE CarTable SET doors = '8' where doors = '2'")
    print('Updated col3(doors) to 8 where ever the value is 2')
    conn.commit()
    
    print('Delete the table')
    cursor.execute("Drop table CarTable")
    conn.commit()
    cursor.execute('use master')
    cursor.execute('DROP DATABASE cardataset')
    conn.commit()
        
except Exception as ex:
    print(type(ex))  

    